<a href="https://colab.research.google.com/github/Kylehanna/Instacart-Market-Basket-Analysis/blob/main/Notebooks/03_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import os 
from statsmodels.graphics.mosaicplot import mosaic
color = sns.color_palette()

#from library.sb_utils import save_file

import warnings
warnings.filterwarnings("ignore")

In [53]:
from dask.distributed import Client

client = Client(n_workers=4)

distributed.nanny - ERROR - Failed while trying to start worker process: failed to validate _ServerOpts(...).prefix: expected a value of type str, got None of type NoneType
distributed.nanny - ERROR - Failed while trying to start worker process: failed to validate _ServerOpts(...).prefix: expected a value of type str, got None of type NoneType
distributed.nanny - ERROR - Failed while trying to start worker process: failed to validate _ServerOpts(...).prefix: expected a value of type str, got None of type NoneType
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
tornado.application - ERROR - Multiple exceptions in yield list
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tornado/gen.py", line 883, in callback
    result_list.append(f.result())
  File "/usr/local/lib/python3.7/dist-packages/tornado/gen.py", line 1147, in run
    yielded = self.gen.send(value)


TypeError: ignored

In [ ]:
import os
import dask
filename = os.path.join('data', 'accounts.*.csv')
filename

In [ ]:
import dask.dataframe as dd
df = dd.read_csv(filename)
df.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = dd.read_csv('/content/drive/MyDrive/GROUND GAME AI/00 Capstone 2 - Instacart Market Basket Analysis (Customer Segmentation)/Instacart-Market-Basket-Analysis/data/instacartData_prior.csv')

In [ ]:
df.head()

In [ ]:
#top_users = df.user_id.value_counts().values[:300000]

In [ ]:
#df_temp = df[df['user_id'].isin(top_users)]

In [ ]:
#df_temp.shape

In [ ]:
#df_temp.user_id.nunique()

In [ ]:
#df_temp.info()

In [ ]:
df.describe()

In [ ]:
df.eval_set.unique()

In [ ]:
df.shape

## Feature Engineering

In [ ]:
#Total number of orders placed by each users
users = df.groupby(by='user_id')['order_number'].aggregate('max').to_frame('u_num_of_orders').reset_index()

In [ ]:
users.head()

In [ ]:
# Average number of products bought by the user in each purchase.
#1. First getting the total number of products in each order.
total_prd_per_order = df.groupby(by=['user_id', 'order_id'])['product_id'].aggregate('count').to_frame('total_products_per_order').reset_index()
total_prd_per_order.head(10)

In [ ]:
#2. Getting the average number of products purchased by each user
avg_products = total_prd_per_order.groupby(by=['user_id'])['total_products_per_order'].mean().to_frame('u_avg_prd').reset_index()
avg_products.head()

In [ ]:
#deleting the total_prd_per_order dataframe
del total_prd_per_order

In [ ]:
#dow of most orders placed by each user
from scipy import stats
dow = df.groupby(by='user_id')['order_dow'].agg(lambda x: stats.mode(x)[0]).to_frame('dow_most_orders_u').reset_index()
dow.head()

In [ ]:
#hour of day when most orders placed by each user
from scipy import stats
hod = df.groupby(by='user_id')['order_hour_of_day'].agg(lambda x: stats.mode(x)[0]).to_frame('hod_most_orders_u').reset_index()
hod.head()

In [ ]:
#merging the user created features.

#1. merging avg_products with users
users = users.merge(avg_products, on='user_id', how='left')
#deleting avg_products
del avg_products
users.head()

In [ ]:
#2. merging dow with users.
users = users.merge(dow, on='user_id', how='left')
#deleting dow
del dow
users.head()

In [ ]:
#number of times a product was purchased.
prd = df.groupby(by='product_id')['order_id'].agg('count').to_frame('prd_count_p').reset_index()
prd.head()

In [ ]:
#products reorder ratio.
reorder_p = df.groupby(by='product_id')['reordered'].agg('mean').to_frame('p_reordered_ratio').reset_index()
reorder_p.head()

In [ ]:
#merging the reorder_p with prd
prd = prd.merge(reorder_p, on='product_id', how='left')
#deleting reorder_p
del reorder_p
prd.head()

In [ ]:
#how many times has a user bought the same product?
uxp = df.groupby(by=['user_id', 'product_id'])['order_id'].agg('count').to_frame('uxp_times_bought').reset_index()
uxp.head()

In [ ]:
#reorder ratio of the user for each product.
reorder_uxp = df.groupby(by=['user_id', 'product_id'])['reordered'].agg('mean').to_frame('uxp_reordered_ratio').reset_index()
reorder_uxp.head()

In [ ]:
#merging the two dataframes into one
uxp = uxp.merge(reorder_uxp, on=['user_id', 'product_id'], how='left')
#deleting reorder_uxp
del reorder_uxp
uxp.head()

In [ ]:
#merging users df into uxp
#the dataset includes samples of all eval_set (prior/train/test) data.
data = uxp.merge(users, on='user_id', how='left')
data.head()

In [ ]:
#merging products df into data
data = data.merge(prd, on='product_id', how='left')
data.head()

In [ ]:
del [users, prd, uxp]

In [ ]:
data.shape 

In [ ]:
orders = dd.read_csv('/content/drive/MyDrive/GROUND GAME AI/00 Capstone 2 - Instacart Market Basket Analysis (Customer Segmentation)/Instacart-Market-Basket-Analysis/data/orders.csv')

In [ ]:
orders.eval_set.value_counts()

In [ ]:
orders.head()

In [ ]:
orders.eval_set.unique()

In [ ]:
#keeping only the train and test from eval_set orders dataframe.

#order_products_prior dataset was too big to work with.
#order_products_train(orders.eval_set=train) df was used instead, and enhanaced with FE.
#Use order.eval_set=='prior' to train df to avoid model overfitting.

order_future = orders.loc[((orders.eval_set == 'train') | (orders.eval_set == 'test')), ['user_id', 'eval_set', 'order_id']]
order_future.head()

In [ ]:
order_future.eval_set.unique()

In [32]:
#merging the order_future with the data.
data = order_future.merge(data, on='user_id', how='left', sort='')
data.head()

,user_id,eval_set,order_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,prd_count_p,p_reordered_ratio
0,1,train,1187899,196,10,0.900000,10,5.9,4,35791,0.776480
1,1,train,1187899,10258,9,0.888889,10,5.9,4,1946,0.713772
2,1,train,1187899,10326,1,0.000000,10,5.9,4,5526,0.652009
3,1,train,1187899,12427,10,0.900000,10,5.9,4,6476,0.740735
4,1,train,1187899,13032,3,0.666667,10,5.9,4,3751,0.657158


In [33]:
#preparing the train df.
#using the df['eval_set']=='train' as the training data due to prior dataset size (32M).
data_train = data.loc[(data.eval_set == 'train')]
data_train.head()

,user_id,eval_set,order_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,prd_count_p,p_reordered_ratio
0,1,train,1187899,196,10,0.900000,10,5.9,4,35791,0.776480
1,1,train,1187899,10258,9,0.888889,10,5.9,4,1946,0.713772
2,1,train,1187899,10326,1,0.000000,10,5.9,4,5526,0.652009
3,1,train,1187899,12427,10,0.900000,10,5.9,4,6476,0.740735
4,1,train,1187899,13032,3,0.666667,10,5.9,4,3751,0.657158


In [ ]:
#merging the information from the order_products_train df into the data_train.
data_train = data_train.merge(df[['product_id', 'order_id', 'reordered']], on=['product_id', 'order_id'], how='left')
data_train.head()

In [86]:
#filling the NAN values
data_train.reordered.fillna(0, inplace=True)

In [87]:
#setting user_id and product_id as index.
data_train = data_train.set_index(['user_id', 'product_id'])

#deleting eval_set, order_id as they are not needed for training.
data_train.drop(['eval_set', 'order_id'], axis=1, inplace=True)

In [88]:
data_train.head()

uxp_times_bought  ...  reordered
user_id product_id                    ...           
1       49235.0                  1.0  ...        1.0
2       22825.0                  1.0  ...        1.0
5       21413.0                  1.0  ...        1.0
8       31717.0                  1.0  ...        0.0
        47766.0                  1.0  ...        0.0

[5 rows x 8 columns]

In [96]:
data_train.isnull().sum()

uxp_times_bought       2962
uxp_reordered_ratio    2962
u_num_of_orders        2962
u_avg_prd              2962
dow_most_orders_u      2962
dtype: int64

In [98]:
#preparing the test dataset.
data_test = data[data.eval_set == 'test']
data_test.head()

,user_id,eval_set,order_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,prd_count_p,p_reordered_ratio
42,3,test,2774568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,4,test,329954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,6,test,1528013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,11,test,1376945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,12,test,1356845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
data_test.isna().sum()

user_id                    0
eval_set                   0
order_id                   0
product_id             75000
uxp_times_bought       75000
uxp_reordered_ratio    75000
u_num_of_orders        75000
u_avg_prd              75000
dow_most_orders_u      75000
prd_count_p            75000
p_reordered_ratio      75000
dtype: int64

In [100]:
#deleting unwanted columns
data_test.drop(['eval_set', 'order_id'], axis=1, inplace=True)

In [101]:
#setting user_id and product_id as index.
data_train = data_train.set_index(['user_id', 'product_id'])

#deleting eval_set, order_id as they are not needed for training.
data_train.drop(['eval_set', 'order_id'], axis=1, inplace=True)

In [102]:
data_test.head()

,user_id,product_id,uxp_times_bought,uxp_reordered_ratio,u_num_of_orders,u_avg_prd,dow_most_orders_u,prd_count_p,p_reordered_ratio
42,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
#deleting unwanted df
del [aisles, departments, order_products_prior, order_products_train, orders, order_future, data] #order_future, data]

NameError: ignored

In [ ]:
#resetting index
data_train.reset_index(inplace=True)
data_test.reset_index(inplace=True)

In [ ]:
data_train.head()

In [ ]:
#merging the aisles and department ids to with the train and test data
data_train = data_train.merge(products[['product_id', 'aisle_id']], on='product_id', how='left')
data_test = data_test.merge(products[['product_id', 'aisle_id']], on='product_id', how='left')

In [ ]:
#department
data_train = data_train.merge(products[['product_id', 'department_id']], on='product_id', how='left')
data_test = data_test.merge(products[['product_id', 'department_id']], on='product_id', how='left')

In [ ]:
#setting user_id and product_id as index.
data_test = data_test.set_index(['user_id', 'product_id'])
#setting user_id and product_id as index.
data_train = data_train.set_index(['user_id', 'product_id'])

In [ ]:
y_train = data_train.reordered
X_train = data_train.drop(columns=['reordered'])

# Save preprocessed data in new csv

In [ ]:
'''# save the data to a new csv file
#datapath = '../../data'
#save_file(df, 'processed.csv', datapath)

df.to_csv('processed_data.csv',index=False)'''